# Project: Evaluating Deep-Learning Models for Detecting Orientation and Flipping of X-Ray Scans
## Objective: Develop a Reliable Method to Detect Orientation and Flipping of PA and Lateral Ceph Scans in the BB-Collection
---
### Background

In the BB-Collection, we identified several TIFF scans with inconsistent rotation and flipping. Specifically, Lateral Cephs and PAs should be oriented to the right and upwards, respectively.

### Strategy

Based on the findings in the work of Swami et al. [[1]](#References), we've determined that deep learning is the most appropriate method to accomplish this task.  
Specifically, we will make use of pre-trained models through *transfer learning*. This is the process of fine-tuning a model that has already been trained on millions of images to recognize more than 1000 classes of objects.  

We will evaluate various models with different architectures and sizes (number of neurons and layers), and assess their performance specifically within the context of the BB-Collection.

In [2]:
import fastai

---
### References
[1]: K. Swami, P. P. Deshpande, G. Khandelwal and A. Vijayvargiya, "Why my photos look sideways or upside down? Detecting canonical orientation of images using convolutional neural networks," 2017 IEEE International Conference on Multimedia & Expo Workshops (ICMEW), Hong Kong, China, 2017, pp. 495-500, doi: 10.1109/ICMEW.2017.8026216.